# Prepare your custom data

## Feature Preparation
- Initial data preparation for E3 ligase and POI, PROTAC
- The raw data format for E3 ligase and target proteins is amino acid sequences
- The raw data format for PROTAC is SMILES

In [1]:
import os
import os.path as osp
import pandas as pd

root = 'data/custom'
os.makedirs(root, exist_ok=True)

In [2]:
raw_df = pd.read_csv(osp.join(root, 'raw.csv'))
raw_df

,Uniprot,E3 ligase Uniprot,Smiles
0,Q9H8M2,P40337,COC1=CC(C2=CN(C)C(=O)C3=CN=CC=C23)=CC(OC)=C1CN...
1,P03372,Q96SW2,CCN(CCCCCCC#CC1=CC=CC2=C1CN(C1CCC(=O)NC1=O)C2=...
2,P10275,P40337,CC1=C(C2=CC=C(CNC(=O)[C@@H]3C[C@@H](O)CN3C(=O)...
3,P10275,P40337,CC1=NOC([C@H](C(=O)N2C[C@H](O)C[C@H]2C(=O)N[C@...
4,P10275,P40337,CC1=C(C2=CC=C([C@H](CC(=O)N3CCC(N4CCC(C#CC5=CC...
5,O60885,Q96SW2,CC1=C(C)C2=C(S1)N1C(C)=NN=C1[C@H](CC(=O)NCCN1C...
6,O15264,P40337,COC1=CC2=C(OC3=CC=C(NC(=O)C4(C(=O)NC5=CC=C(F)C...
7,Q16539,P40337,COC1=CC2=C(OC3=CC=C(NC(=O)C4(C(=O)NC5=CC=C(F)C...
8,P11474,P40337,COC1=CC(/C=C(\C#N)C(=O)NCCCC(=O)N[C@H](C(=O)N2...
9,O60885,P40337,CC1=C(C2=CC=C(CNC(=O)[C@@H]3C[C@@H](O)CN3C(=O)...


### Prepare PROTAC

In [3]:
columns = [
    # 'Smiles',
    'Molecular Weight',
    'Exact Mass',
    'XLogP3',
    'Heavy Atom Count',
    'Ring Count',
    'Hydrogen Bond Acceptor Count',
    'Hydrogen Bond Donor Count',
    'Rotatable Bond Count',
    'Topological Polar Surface Area'
]

Calculate molecular properties for PROTAC

> [!NOTE]
> `XLogP3` is originally calculated via [XLogP3](http://www.sioc-ccbg.ac.cn/skins/ccbgwebsite/software/xlogp3/) software, here using `Descriptors.MolLogP()` from [RDKit](https://www.rdkit.org/)

In [4]:

from rdkit import Chem
from rdkit.Chem import Descriptors


def calculate_molecular_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return [
        Descriptors.MolWt(mol),
        Descriptors.ExactMolWt(mol),
        Descriptors.MolLogP(mol), # XLogP3 to LogP
        Descriptors.HeavyAtomCount(mol),
        Descriptors.RingCount(mol),
        Descriptors.NumHAcceptors(mol),
        Descriptors.NumHDonors(mol),
        Descriptors.NumRotatableBonds(mol),
        Descriptors.TPSA(mol)
    ]

In [5]:
property_df = raw_df['Smiles'].apply(calculate_molecular_descriptors).apply(pd.Series)
property_df.columns = columns
custom_df = pd.concat([raw_df, property_df], axis=1)
custom_df

,Uniprot,E3 ligase Uniprot,Smiles,Molecular Weight,Exact Mass,XLogP3,Heavy Atom Count,Ring Count,Hydrogen Bond Acceptor Count,Hydrogen Bond Donor Count,Rotatable Bond Count,Topological Polar Surface Area
0,Q9H8M2,P40337,COC1=CC(C2=CN(C)C(=O)C3=CN=CC=C23)=CC(OC)=C1CN...,1041.257,1040.484140,4.89042,74.0,8.0,16.0,3.0,22.0,199.15
1,P03372,Q96SW2,CCN(CCCCCCC#CC1=CC=CC2=C1CN(C1CCC(=O)NC1=O)C2=...,783.947,783.297822,7.67440,57.0,7.0,9.0,3.0,15.0,136.48
2,P10275,P40337,CC1=C(C2=CC=C(CNC(=O)[C@@H]3C[C@@H](O)CN3C(=O)...,1047.255,1046.416987,9.11460,73.0,6.0,10.0,4.0,20.0,188.07
3,P10275,P40337,CC1=NOC([C@H](C(=O)N2C[C@H](O)C[C@H]2C(=O)N[C@...,1114.302,1113.422800,9.58712,78.0,7.0,12.0,4.0,22.0,214.10
4,P10275,P40337,CC1=C(C2=CC=C([C@H](CC(=O)N3CCC(N4CCC(C#CC5=CC...,1136.861,1135.512044,8.35418,81.0,9.0,12.0,4.0,14.0,221.09
5,O60885,Q96SW2,CC1=C(C)C2=C(S1)N1C(C)=NN=C1[C@H](CC(=O)NCCN1C...,913.414,912.278022,3.62756,64.0,8.0,17.0,2.0,19.0,223.35
6,O15264,P40337,COC1=CC2=C(OC3=CC=C(NC(=O)C4(C(=O)NC5=CC=C(F)C...,1078.205,1077.411783,7.71442,77.0,8.0,14.0,5.0,23.0,228.87
7,Q16539,P40337,COC1=CC2=C(OC3=CC=C(NC(=O)C4(C(=O)NC5=CC=C(F)C...,1180.341,1179.458733,10.39662,85.0,10.0,14.0,4.0,26.0,220.08
8,P11474,P40337,COC1=CC(/C=C(\C#N)C(=O)NCCCC(=O)N[C@H](C(=O)N2...,942.980,942.320938,7.35620,66.0,5.0,10.0,4.0,16.0,182.98
9,O60885,P40337,CC1=C(C2=CC=C(CNC(=O)[C@@H]3C[C@@H](O)CN3C(=O)...,1092.786,1091.416430,8.65168,76.0,8.0,15.0,4.0,22.0,211.49


In [6]:
custom_df.to_csv(osp.join(root, 'custom.csv'), index=False)

### Prepare E3 ligase and target

In [7]:
# collect all uniprot ids
uniprot_ids = set(custom_df['Uniprot'].tolist() + custom_df['E3 ligase Uniprot'].tolist())
print(uniprot_ids)

{'Q96SW2', 'O15264', 'P10275', 'P11474', 'P40337', 'Q9H8M2', 'Q16539', 'O60885', 'P03372'}


In [8]:
import pickle
import time
from io import StringIO

import requests
from Bio import SeqIO
from requests.exceptions import HTTPError, RequestException

try: 
    seq_cache = pickle.load(open(osp.join(root, 'seq_cache.pkl'), 'rb'))
except:
    seq_cache = {}

def get_aa_seq(uniprot_id):
    """
    Download fasta file from Uniprot and extract amino acid sequence
    
    Parameters:
        uniprot_id (str): UniProt ID of the protein.
        
    Returns:
        str: Amino acid sequence of the protein.
        
    Example:
        get_aa_seq('P04637')
        >>> 'MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLSPDDIEQWFTEDPGPDEAPRMPEAAPPVAPAPAAPTPAAPAPAPSWPLSSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENLRKKGEPHHELPPGSTKRALPNNTSSSPQPKKKPLDGEYFTLQIRGRERFEMFRELNEALELKDAQAGKEPGGSRAHSSHLKSKKGQSTSRHKKLMFKTEGPDSD'
    """
    try:
        if uniprot_id in seq_cache:
            return seq_cache[uniprot_id]
        
        url = f'https://www.uniprot.org/uniprot/{uniprot_id}.fasta'
        r = requests.get(url)
        r.raise_for_status()
        fasta = r.text
        fasta_io = StringIO(fasta)
        seq_record = SeqIO.read(fasta_io, 'fasta')
        time.sleep(1)
        seq = str(seq_record.seq)
        seq_cache[uniprot_id] = seq
        pickle.dump(seq_cache, open(osp.join(root, 'seq_cache.pkl'), 'wb'))
        return seq
    except HTTPError as http_err:
        print(f'发生HTTP错误: {http_err}')
    except RequestException as req_err:
        print(f'发生请求错误: {req_err}')
    except Exception as e:
        print(f'发生意外错误: {e}')

In [9]:
p_map = {uniprot_id: get_aa_seq(uniprot_id) for uniprot_id in uniprot_ids}
for k, v in p_map.items():
    print(k, v)

Q96SW2 MAGEGDQQDAAHNMGNHLPLLPAESEEEDEMEVEDQDSKEAKKPNIINFDTSLPTSHTYLGADMEEFHGRTLHDDDSCQVIPVLPQVMMILIPGQTLPLQLFHPQEVSMVRNLIQKDRTFAVLAYSNVQEREAQFGTTAEIYAYREEQDFGIEIVKVKAIGRQRFKVLELRTQSDGIQQAKVQILPECVLPSTMSAVQLESLNKCQIFPSKPVSREDQCSYKWWQKYQKRKFHCANLTSWPRWLYSLYDAETLMDRIKKQLREWDENLKDDSLPSNPIDFSYRVAACLPIDDVLRIQLLKIGSAIQRLRCELDIMNKCTSLCCKQCQETEITTKNEIFSLSLCGPMAAYVNPHGYVHETLTVYKACNLNLIGRPSTEHSWFPGYAWTVAQCKICASHIGWKFTATKKDMSPQKFWGLTRSALLPTIPDTEDEISPDKVILCL
O15264 MSLIRKKGFYKQDVNKTAWELPKTYVSPTHVGSGAYGSVCSAIDKRSGEKVAIKKLSRPFQSEIFAKRAYRELLLLKHMQHENVIGLLDVFTPASSLRNFYDFYLVMPFMQTDLQKIMGMEFSEEKIQYLVYQMLKGLKYIHSAGVVHRDLKPGNLAVNEDCELKILDFGLARHADAEMTGYVVTRWYRAPEVILSWMHYNQTVDIWSVGCIMAEMLTGKTLFKGKDYLDQLTQILKVTGVPGTEFVQKLNDKAAKSYIQSLPQTPRKDFTQLFPRASPQAADLLEKMLELDVDKRLTAAQALTHPFFEPFRDPEEETEAQQPFDDSLEHEKLTVDEWKQHIYKEIVNFSPIARKDSRRRSGMKL
P10275 MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAASAAPPGASLLLLQQQQQQQQQQQQQQQQQQQQQQQETSPRQQQQQQGEDGSPQAHRRGPTGYLVLDEEQQPSQPQSALECHPERGCVPEPGAAVAASKGLPQQLPAPPDEDDSAAPSTLSLLGPT

In [10]:
print(root)

data/custom


In [11]:
with open(osp.join(root, 'p_map.pkl'), 'wb') as f:
    pickle.dump(p_map, f)

Processing protein sequence features into implicit structural representations
- Protein sequence hash table: `p_map`
- Protein structural representation hash table: `esm_s_map`

> ![NOTE] 
> This part needs to be prepared separately, taking into account the large GPU resources required for the Protein Language Model as well as persistent data to improve training efficiency. 
```shell
$ conda activate ems+
$ python get_embed_s.py
```

## Ready for PROTAC-STAN inference

- Your ready custom data: `customed.csv` and `esm_s_map.pkl`, keep them in the same directory, such as: `./data/custom`
- Using `inference.py`, usage: `python inference.py [-h] [--root ROOT] [--name NAME] [--save_att]`
  - for example:
    ```shell
    $ conda activate protac-stan
    $ python inference.py --root 'data/custom' --name 'custom'
    ```